In [1]:
#Required Modules---

import boto.ec2
import boto.ec2.elb
from boto.ec2.elb import HealthCheck
from boto.ec2.autoscale import LaunchConfiguration
from boto.ec2.autoscale import AutoScalingGroup
from boto.ec2.autoscale import ScalingPolicy
from boto.ec2.cloudwatch import MetricAlarm
import boto.ec2.cloudwatch

In [2]:
#script for install apahche server and copy files from s3 bucket

fh = open('apache_script.sh')
s = ''
for lines in fh:
	s += lines

In [4]:
#connection for autoscale and creation of launch configuration for autoscaling

conn=boto.ec2.autoscale.connect_to_region("us-east-2",aws_access_key_id='AKIAxxxxxxxxxxxxxxx',aws_secret_access_key='MMbLhCzdmL9Kxxxxxxxxxxxxxxxxxxxx')
lc = LaunchConfiguration(name='my-launch_config_01', image_id= 'ami-07c8bc5c1ce9598c3',
                             key_name='dwarka',instance_type='t2.micro',user_data=s,
                             security_groups=['sg-01f0b320d6d41edf3'])
conn.create_launch_configuration(lc)

Request:d4010627-5cdb-4a5b-9838-ef19205f6f1d

In [5]:
#creating a autoscaling group with load_balancer

ag=AutoScalingGroup(group_name='boto_group_1',availability_zones=['us-east-2a','us-east-2b','us-east-2c'],launch_config=lc,min_size=1,max_size=4,connection=conn)
#load_balancers=['my-lb'],
conn.create_auto_scaling_group(ag)

Request:b2d09796-6d3f-4cac-bd41-24b3792f1287

In [6]:
#getting all existing autoscale group

conn.get_all_groups()

[AutoScaleGroup<boto_group>, AutoScaleGroup<boto_group_1>]

In [7]:
#activites

conn.get_all_activities(ag)

[]

In [8]:
#Scaling a Group Up or Down


scale_up_policy = ScalingPolicy(
            name='scale_up', adjustment_type='ChangeInCapacity',
            as_name='boto_group_1', scaling_adjustment=1, cooldown=180)
scale_down_policy = ScalingPolicy(
            name='scale_down', adjustment_type='ChangeInCapacity',
            as_name='boto_group_1', scaling_adjustment=-1, cooldown=180)

In [9]:
#submiting aws policies to aws

conn.create_scaling_policy(scale_up_policy)
conn.create_scaling_policy(scale_down_policy)

Request:27e7c1a8-54e4-452f-9eed-212a76a2df94

In [10]:
#getting ARN of each policies

scale_up_policy = conn.get_all_policies(
            as_group='boto_group_1', policy_names=['scale_up'])[0]
scale_down_policy = conn.get_all_policies(
            as_group='boto_group_1', policy_names=['scale_down'])[0]

In [11]:
#Alarm to run the autoscaling policies

cloudwatch = boto.ec2.cloudwatch.connect_to_region('us-east-2')
alarm_dimensions = {"AutoScalingGroupName": 'boto_group_1'}

In [12]:
#scale_up alarm

scale_up_alarm = MetricAlarm(
            name='scale_up_on_cpu', namespace='AWS/EC2',
            metric='CPUUtilization', statistic='Average',
            comparison='>', threshold='70',
            period='60', evaluation_periods=2,
            alarm_actions=[scale_up_policy.policy_arn],
            dimensions=alarm_dimensions)


cloudwatch.create_alarm(scale_up_alarm)

True

In [13]:
#scale_down alarm

scale_down_alarm = MetricAlarm(
            name='scale_down_on_cpu', namespace='AWS/EC2',
            metric='CPUUtilization', statistic='Average',
            comparison='<', threshold='40',
            period='60', evaluation_periods=2,
            alarm_actions=[scale_down_policy.policy_arn],
            dimensions=alarm_dimensions)

cloudwatch.create_alarm(scale_down_alarm)

True

In [14]:
#getting instances belonging to out autoscale group and getting their public DNS name

ec2 = boto.ec2.connect_to_region('us-east-2')
group = conn.get_all_groups(names=['boto_group_1'])[0]
instance_ids = [i.instance_id for i in group.instances]
instances = ec2.get_only_instances(instance_ids)
for i in instances:
    print(i.public_dns_name)

ec2-18-188-229-202.us-east-2.compute.amazonaws.com


In [15]:
#shutdown all autoscale instances

ag.shutdown_instances()


In [16]:
#delete autoscale group

ag.delete()

Request:ec4e9f51-4a85-4e8f-aa4c-a750503ef7ce